In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.utils import to_categorical


In [3]:
FEATURE_DIR = "E:/voiceguardian/data/features"
emotion_map = {
    "01": "neutral", "02": "calm", "03": "happy", "04": "sad",
    "05": "angry", "06": "fearful", "07": "disgust", "08": "surprised"
}

X, y = [], []
for file in os.listdir(FEATURE_DIR):
    if file.endswith(".npy"):
        mfcc = np.load(os.path.join(FEATURE_DIR, file))
        if mfcc.shape[0] >= 60:  # Ensure minimum length
            mfcc = mfcc[:60]  # Truncate
            X.append(mfcc)
            label = emotion_map.get(file.split("-")[2], "unknown")
            y.append(label)


In [4]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

X = np.array(X)  # Shape: (samples, timesteps, features)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)


In [6]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking

model = Sequential([
    Input(shape=(60, 40)),                 # 👈 Correct way to specify input
    Masking(mask_value=0.),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_cat.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape          ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ masking_1 (Masking)           │ (None, 60, 40)        │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ lstm_1 (LSTM)                 │ (None, 128)           │       86,528 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dropout_2 (Dropout)           │ (None, 128)           │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_2 (Dense)               │ (None, 64)            │        8,256 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dropout_3 (Dropout)           │ (None, 64)            │            0 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_3 (Dense)               │ (None, 8)             │          520 │
└───────────────────────────────┴───────────────────────┴──────────────┘

 Total params: 95,304 (372.28 KB)

 Trainable params: 95,304 (372.28 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_split=0.2, verbose=1)


Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 205ms/step - accuracy: 0.1137 - loss: 2.2044 - val_accuracy: 0.2432 - val_loss: 1.9994
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 154ms/step - accuracy: 0.2197 - loss: 2.0227 - val_accuracy: 0.3018 - val_loss: 1.9090
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.2493 - loss: 1.9460 - val_accuracy: 0.3108 - val_loss: 1.8564
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 165ms/step - accuracy: 0.2578 - loss: 1.8922 - val_accuracy: 0.3063 - val_loss: 1.7851
Epoch 5/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 159ms/step - accuracy: 0.2953 - loss: 1.8248 - val_accuracy: 0.3333 - val_loss: 1.7449
Epoch 6/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 142ms/step - accuracy: 0.3344 - loss: 1.7744 - val_accuracy: 0.3423 - val_loss: 1.6923
Epoch 7/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 166ms/step - accuracy: 0.3859 - loss: 1.6584 - val_accuracy: 0.3649 - val_loss: 1.6765
Epoch 8/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 154ms/step - accuracy: 0.3865 - loss: 1.6670 - val_accuracy: 0

In [10]:
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.2f}")

model.save("E:/voiceguardian/models/emotion_lstm_model.keras")
np.save("E:/voiceguardian/models/label_classes.npy", le.classes_)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.3697 - loss: 2.3214
✅ Test Accuracy: 0.40


In [11]:
streamlit run app.py

E:\voiceguardian\voice_env\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
